In [2]:
pip install torchmetrics

In [57]:
pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 8.6 MB/s eta 0:00:00


In [3]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [58]:
from torchmetrics.multimodal.clip_score import CLIPScore
from PIL import Image
import torch
import os
from datasets import load_dataset
from datasets.utils.file_utils import get_datasets_user_agent
import requests
import numpy as np
from natsort import natsorted
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util

USER_AGENT = get_datasets_user_agent()


In [5]:


def calculate_clip_score(images, prompts):
    metrics = CLIPScore(model_name_or_path="openai/clip-vit-base-patch16")
    images_int = (images * 255).astype("uint8")
    images_int = np.expand_dims(images_int, axis=0)  # Add batch dimension
    score = metrics(torch.from_numpy(images_int).permute(0, 3, 1, 2), prompts).detach()
    return round(float(score), 4)


In [67]:
def calculate_blip_2(image, prompt, processor ,model,device):
    try:
        img_inputs = processor(images=image, return_tensors="pt").to(device)

        # Generate image features
        with torch.no_grad():
          generated_ids = model.generate(**img_inputs)

        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
        similarity_model = SentenceTransformer('all-MiniLM-L6-v2')
        generated_text_embedding = similarity_model.encode(generated_text)
        caption_embedding = similarity_model.encode(prompt)

        similarity_score = util.pytorch_cos_sim(caption_embedding, generated_text_embedding).item()
        return similarity_score, generated_text
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

Data set & captions


In [8]:
num_threads = 20
dset = load_dataset("google-research-datasets/conceptual_captions",streaming=True)


README.md:   0%|          | 0.00/14.2k [00:00<?, ?B/s]

In [9]:
dset_iter = iter(dset['train'])

In [19]:

def conceptual_captioning_stream_to_dir (save_dir, image_num):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    captions = []
    for i in range(image_num):
        item2 = next(dset_iter)
        image_url = item2['image_url']
        caption = item2['caption']
        response = requests.get(image_url, stream=True)
        try:
            if response.status_code == 200:
                image_path = os.path.join(save_dir, f"image_{i}.jpg")
                with open(image_path, 'wb') as f:
                    for chunk in response.iter_content(1024):
                        f.write(chunk)
                captions.append(caption)
            else:
                print(f"Failed to download image {i} from {image_url}")
        except Exception as e:
            print(f"Error downloading image {i} from {image_url}: {e}")
    return captions , save_dir

In [68]:
def evaluation(save_dir, captions , metric= "clip"):
    image_files = natsorted(os.listdir(save_dir))  # Ensure the images are processed in order
    if (metric == "clip"):
        print(f"Metric used is CLIP Score")
        for img_file, caption in zip(image_files, captions):
            img_path = os.path.join(save_dir, img_file)
            if img_path.endswith(".jpg") or img_path.endswith(".png"):
                img = Image.open(img_path)
                img_array = np.array(img) / 255.0
                score = calculate_clip_score(img_array, caption)
                print(f"Image: {img_path}, Caption: {caption}, CLIP Score: {score}")
    elif(metric == "blip"):
        print(f"Metric used is BLIP Score")
        processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xl")
        model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-flan-t5-xl")
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using {device}")
        model.to(device)
        for img_file, caption in zip(image_files, captions):
            img_path = os.path.join(save_dir, img_file)
            if img_path.endswith(".jpg") or img_path.endswith(".png"):
                img = Image.open(img_path)
                simularity, generated_caption = calculate_blip_2(img,caption,processor,model,device)
                print(f"Image: {img_path}, Caption: {caption}, Generated Caption {generated_caption} BLIP2 Score: {simularity}")



Download Images to directory

In [24]:
# remove before merge

captions = []
path_dir = os.path.join(os.getcwd(), "conceptual_captioning")
captions, save_dir = conceptual_captioning_stream_to_dir(path_dir,image_num=20)


Clip Score Evaluation


In [23]:
# remove before merge
import shutil

image_path = os.path.join(os.getcwd(), "conceptual_captioning")

if os.path.exists(image_path):
    shutil.rmtree(image_path)
    print(f"{image_path} has been deleted.")
else:
    print(f"{image_path} does not exist.")

/content/conceptual_captioning has been deleted.


In [26]:
evaluation(save_dir,captions)

Metric used is CLIP Score


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Image: /content/conceptual_captioning/image_0.jpg, Caption: actor and daughters uk premiere held, CLIP Score: 30.4291
Image: /content/conceptual_captioning/image_1.jpg, Caption: a fine , grainy vector pattern in black and white ., CLIP Score: 29.4877
Image: /content/conceptual_captioning/image_2.jpg, Caption: seamless border of orange roses and paisley , pattern on a white background ., CLIP Score: 29.311
Image: /content/conceptual_captioning/image_3.jpg, Caption: students in front of a school, CLIP Score: 28.599
Image: /content/conceptual_captioning/image_4.jpg, Caption: museum , opened is among the 20th - century 's most significant buildings ., CLIP Score: 27.0254
Image: /content/conceptual_captioning/image_5.jpg, Caption: river as it meets bodies of water where airline meets, CLIP Score: 25.7665
Image: /content/conceptual_captioning/image_6.jpg, Caption: young rock star jamming on a guitar, CLIP Score: 27.6241
Image: /content/conceptual_captioning/image_7.jpg, Caption: a moored fis

BLIP2 evaluation

In [69]:
evaluation(save_dir,captions, metric='blip')

Metric used is BLIP Score


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Using cpu


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Image: /content/conceptual_captioning/image_0.jpg, Caption: actor and daughters uk premiere held, Generated Caption samuel l. jackson and his family at the uk BLIP2 Score: 0.28897207975387573
Image: /content/conceptual_captioning/image_1.jpg, Caption: a fine , grainy vector pattern in black and white ., Generated Caption black and white seamless pattern with a pattern of leaves BLIP2 Score: 0.5404555201530457
Image: /content/conceptual_captioning/image_2.jpg, Caption: seamless border of orange roses and paisley , pattern on a white background ., Generated Caption watercolor floral border with roses and paisley BLIP2 Score: 0.8097856640815735
Image: /content/conceptual_captioning/image_3.jpg, Caption: students in front of a school, Generated Caption a group of school children posing for a photo in a school - BLIP2 Score: 0.6507741212844849
Image: /content/conceptual_captioning/image_4.jpg, Caption: museum , opened is among the 20th - century 's most significant buildings ., Generated Ca